## Procesamiento de distintos Datasets de religiones en el mundo
El objetivo de este procesamiento de datos es limpiar, integrar y transformar los datos de distintos set de datos para posteriormente poder realizar un analisís mediante visualizaciones realizadas con SVG.


Para esto utilizaré la libreria Pandas de Python para el procesamiento de los datos que se encuentran en formato CSV y JSON


In [1]:
import pandas as pd
import json
from heapq import nlargest
from pandas.io.json import json_normalize

from IPython.display import display

Vamos a crear los distintos dataframes para poder generar los archivos a utilizar en el punto2. 


Creamos los dataframes con los distintos datasets:

In [2]:
wrp_global = pd.read_csv('data/WRP_global.csv')
wrp_nation = pd.read_csv('data/WRP_national.csv')
wrp_region = pd.read_csv('data/WRP_regional.csv')
wrp_COW = pd.read_csv('data/COW_country_codes.csv')

with open('data/dataset_countries.json') as json_data:
    countries = json.load(json_data)
    json_data.close()



## Heatmap
La preparación de los datos para esta visualización consiste en varios pasos que detallaré a continuacion:
1. Determinar las religiones mas numerosas en funcion de el dataset WRP_global, de donde determinare que religiones van a estar en el heatmap.
2. Cruzar datos entre los paises que están en **WRP_national** y la lista de paises según continentes que se encuentra en **dataset_json**. De manera de determinar a que continente pertenecen los paises que estan en **WRP_national**. De esto va a salir un nuevo dataframe de **Pandas** que va a contener a los paises según continente.
Ya que la información que se tiene en **WRP_regional** no incluye a America y Oceania especificamente como tal.  
3. Del dataframe anterior contamos la cantidad total de personas por continente. 
4. Procedemos a calcular la cantidad de practicantes por religion por continente según la lista de religiones obtenida en (2)


**Importante**: Esta selección la haré en función de los últimos datos obtenidos en 2010.

Empezamos la preparación de datos siguiendo el esquema anterior.


Religiones más numerosas:

In [9]:
# Seleccionamos la fila correspondiente al año 2010 y solo columnas con el numero de practicantes por religion
global_religions = wrp_global[wrp_global['year'] == 2010].as_matrix()[0][0:36]

# cambiamos las comas por espacios vacios para castear a float y obtener maximos
global_religions = list(map(lambda x: float(x.replace(',','')) if type(x) == str else float(x), global_religions))

# nombre de las 10 religiones mas numerosas en orden descendiente de izquierda a derecha
largest_religions = []
for i in nlargest(10, global_religions):
    for j in range(len(global_religions)):
        if i == global_religions[j]:
            largest_religions.append(wrp_global.columns[j])
print("Religiones con más practicantes a nivel mundial:\n{}".format(largest_religions))

Religiones con más practicantes a nivel mundial:
['chrstgen', 'islmgen', 'islmsun', 'chrstcat', 'hindgen', 'nonrelig', 'syncgen', 'chrstprot', 'budgen', 'budothr']


Ahora necesitamos cruzar datos entre los dataset WRP_national y dataset_json, de manera de tener la correlacion pais-continente.

Partimos relacionando pais-continente:

In [111]:
# CONTINENTES Y SUS RESPECTIVOS PAISES
continentes = {'europe': list(map(lambda x: x['name']['common'], countries['Europe'])),
               'america': list(map(lambda x: x['name']['common'], countries['Americas'])),
               'asia': list(map(lambda x: x['name']['common'], countries['Asia'])),
               'oceania': list(map(lambda x: x['name']['common'], countries['Oceania'])),
               'africa': list(map(lambda x: x['name']['common'], countries['Africa']))}

# transformamos la lista de paises en diccionarios para poder agregar caracteristicas
for continente in continentes:
    aux = {}
    for pais in continentes[continente]:
        aux[pais] = {}
    continentes[continente] = aux

count = 0
for continente in continentes:
    for pais in continentes[continente]:
        count += 1
        
# Ocupamos el dataset auxiliar COW_country_codes y asociamos el codigo con el nombre del pais
code_abr = wrp_nation[wrp_nation['year'] == 2010].ix[:,1:3]

# tenemos 194 paises con datos del año 2010 y 148 paises en el dataset_countries.json
# revisamos si 46 paises paises de diferencia son realmente significativos en el porcentaje total de problación

# Utilizamos el dataset de codigos de paises
code_name = code_abr.join(wrp_COW.set_index(['CCode']), on='state')

# Según los 194 países en WRP_nation, al año 2010 existe una poblacion total de:
wrp_total_pop = wrp_nation[wrp_nation['year'] == 2010]['pop'].sum()

# Consideramos los datos del año 2010 de los paises 
wrp_countries_2010 = wrp_nation[wrp_nation['year'] == 2010]

# agregamos a nuestro diccionario de continentes el codigo de pais correspondiente, según el dataset auxiliar
for index, row in code_name.iterrows():
    for continente, paises in continentes.items():
        for name, value in paises.items():
            if name in row['StateNme']:
                continentes[continente][name]['state_code'] = row['state']

# cruzamos los datos de poblacion de WRP_national con los paises que tenemos en countries.json
# agregamos informacion relacionada a la poblacion total de cada pais
for index, row in wrp_countries_2010.iterrows():
     for continente, paises in continentes.items():
            for name, value in paises.items():
                if value != {}:
                    if value['state_code'] == row['state']:
                        continentes[continente][name]['pop'] = row['pop']


# buscamos paises de los que no tenemos informacion en WRP_national pero se encuentran en countries.json
no_info = []
no_info_countries = []
for continente, paises in continentes.items():
    for name, values in paises.items():
        if values == {}:
            no_info.append((continente, name))
            no_info_countries.append(name)
print("Paises de los que no tenemos informacion en WRP_nation.csv pero que estan en data_countries.json:\n", no_info)
# eliminamos estos paises
for x in no_info:
    del continentes[x[0]][x[1]]    

# sumamos la cantidad de poblacion total que se tiene en este conjunto reducido de paises
seleccion_total_pop = 0
for continente, paises in continentes.items():
    for name, value in paises.items():
        seleccion_total_pop += value['pop']

# Si comparamos la poblacion total de los paises que seleccionamos en base a la lista data_countries.json y 
# la poblacion total obtenida de WRP_national

print("Poblacion total según WRP_national: {}\n"
      "Poblacion total de mi selección de paises:{}\n"
      "Población total que dejamos fuera: {} ,que equivale a un: {}% de la población según WRP_national\n".format(wrp_total_pop,seleccion_total_pop,
                                                                              wrp_total_pop - seleccion_total_pop,
                                                                              round(100 - (seleccion_total_pop * 100 / wrp_total_pop), 2)))

Paises de los que no tenemos informacion en WRP_nation.csv pero que estan en data_countries.json:
 [('europe', 'Serbia'), ('asia', 'Timor-Leste'), ('asia', 'Palestine'), ('america', 'Saint Lucia')]
Poblacion total según WRP_national: 6830615581
Poblacion total de mi selección de paises:6406584710
Población total que dejamos fuera: 424030871 ,que equivale a un: 6.21% de la población según WRP_national



Tomaré la decisión de dejar fuera este 6% de la población, debido a que de esta manera puedo representar mejor el numero de practicantes por religion por continente. Si hubiese tomado los datos del año 2010 del dataset `WRP_global`, no tendría certeza acerca de que paises componen los continentes mencionados en este dataset. 

Gracias al resultado anterior podemos agregar los practicantes de las diez religiones mas numerosas a cada pais de mi selección y podemos calcular la poblacion total por continente:

In [112]:
for religion in largest_religions:
    for index, row in wrp_countries_2010.iterrows():
         for continente, paises in continentes.items():
                for name, value in paises.items():
                    if value['state_code'] == row['state']:
                        continentes[continente][name][religion] = row[religion]

Ahora voy a calcular la población total por continente: 

In [113]:
pop_per_continent = {}
for continente, paises in continentes.items():
    aux = 0
    for name, value in paises.items():
        aux += value['pop']
    pop_per_continent[continente] = aux
print("Poblacion por continente:\n",pop_per_continent)

Poblacion por continente:
 {'africa': 949596601, 'europe': 717572985, 'asia': 3799998439, 'america': 916165463, 'oceania': 23251222}


Ahora podemos realizar el punto **4** descrito anteriormente y obtenemos la cantidad de practicantes por religion por continente:

In [114]:
religion_per_continent = {'asia':{}, 'america': {}, 'oceania':{}, 'africa':{}, 'europe': {}}
for religion in largest_religions:
    for continente, paises in continentes.items():
        for name, value in paises.items():
            if value['state_code'] == row['state']:
                continentes[continente][name][religion] = row[religion]

for religion in largest_religions:
    for continente, paises in continentes.items():
        aux = 0
        for name, value in paises.items():
            aux += value[religion]
        religion_per_continent[continente][religion] = aux
#         print(aux, religion, continente)     
print("Numero de practicantes por religion por continente:\n",religion_per_continent)

Numero de practicantes por religion por continente:
 {'africa': {'budothr': 211225, 'chrstprot': 143171479, 'syncgen': 105221, 'hindgen': 2241879, 'chrstgen': 461438996, 'budgen': 211225, 'islmgen': 393976657, 'nonrelig': 9317373, 'chrstcat': 140963289, 'islmsun': 374579127}, 'europe': {'budothr': 1556268, 'chrstprot': 65402075, 'syncgen': 506600, 'hindgen': 1124771, 'chrstgen': 541687398, 'budgen': 1556268, 'islmgen': 40297516, 'nonrelig': 121751715, 'chrstcat': 241584171, 'islmsun': 34128120}, 'asia': {'budothr': 273711204, 'chrstprot': 87564561, 'syncgen': 567689622, 'hindgen': 1006258003, 'chrstgen': 242761350, 'budgen': 362144542, 'islmgen': 1022742779, 'nonrelig': 462977583, 'chrstcat': 108420651, 'islmsun': 819675195}, 'oceania': {'budothr': 531866, 'chrstprot': 2857201, 'syncgen': 70333, 'hindgen': 420609, 'chrstgen': 14583581, 'budgen': 531866, 'islmgen': 529549, 'nonrelig': 6659707, 'chrstcat': 4690136, 'islmsun': 0}, 'america': {'budothr': 3494555, 'chrstprot': 235675605, 's

Con esta información ya estoy en condiciones de poder realizar el **Heatmap**

Pero antes de eso, creamos el archivo con la información anterior:

In [125]:

procesed_data = {'Population': pop_per_continent, 'PracticantesPerContinent':religion_per_continent, 'Religions':largest_religions}
with open('processed_data/heatmap.json', 'w') as fp:
    json.dump(procesed_data, fp)

El archivo se compone de dos entidades. La primera de ellas *Population*, contiene la poblacion total por continente. La segunda entidad *PracticantesPerContinent* contiene la cantidad de practicantes por religion por continente. 

## Bubblechart

En esta sección generare los archivos necesarios para realizar los bubblecharts.
Del mismo modo que para el **Heatmap** describiré los pasos a seguir, pero considerando que son dos visualizaciones a genenerar. 


#### Continente como color: 
1. Utilizaré la lista de paises con información que obtuve para hacer el Heatmap
2. Del archivo data_countries.json obtengo la información asociada a continente, población, IDH y coef.Gini
3. Doy formato a esta información y se exporta para realizar el Bubblechart.

**Nota**: A pesar de que toda la información la puedo sacar directamente de data_countries.json, prefiero mantenerme con la selección de paises que realicé para poder tener la mayor consistencia posible


Lista de paises de las que tengo información:

In [130]:
## Seleccionamos los paises de lso que tenemos informacion


# bubblecharts va a contener la data para ambas visualizaciones 
bubblecharts = {}
for continente in countries.keys():
    for countrie in countries[continente]:
        if countrie['name']['common'] not in no_info_countries:
            bubblecharts[countrie['name']['common']]= {'area':countrie['area'], 'hdi': float(countrie['hdi']), 
                     
                                                                     'gini':float(countrie['gini']), 'continente':continente}



Exportaré esta información para usarla en crear el primer bubblechart.

#### Religión mas prácticada como color:
1. Buscar la religión más popular del pais segun los datos en WRP_nation.csv 
2. Utilizaré la información del pimer bubblechart pero reemplazando el continente por la religion.

In [131]:

# best_relig_in_country almacena la religion mas popular del pais
best_relig_in_country = {}
for index, row in wrp_countries_2010.iterrows():
    for continente in continentes.keys():
        for pais,values in continentes[continente].items():
            if values['state_code'] == row['state']:
                # concateno solo los valores correspondientes a las religiones mayores, sin sus familias religiosas
                religions = [row[8], row[13], row[21], row[25], row[26], row[27], row[28], row[29], row[30],
                             row[31], row[32], row[33], row[34], row[35], row[37]]
                biggest_relig = max(religions)
                relig_names = ['Christianity','Judaism', 'Islam', 'Buddhism', 'Zoroastrian', 'Hindu', 'Sikh', 'Shinto',
                               'Baha’i', 'Taoism', 'Confucianism', 'Jain', 'Syncretic', 'Animist', 'Non. Religious',
                               'Other']
                best_relig_in_country[pais] = relig_names[religions.index(biggest_relig)]

# Cruzamos con los datos del json
for country in bubblecharts.keys():
    bubblecharts[country]['religion'] = best_relig_in_country[country]


Logré obtener toda la información que necesitaba y la guarde en el diccionario `bubblecharts`.

Ahora se puede exportar este diccionario a un archivo y usarlo para generar las visualizaciones.

In [132]:
with open('processed_data/bubblecharts.json', 'w') as fp:
    json.dump(bubblecharts, fp)